In [17]:
import taichi as ti
ti.init(arch=ti.cpu)


max_num_particles=1024
x=ti.Vector.field(2,dtype=ti.f32,shape=max_num_particles)
v=ti.Vector.field(2,dtype=ti.f32,shape=max_num_particles)
fixed=ti.field(dtype=ti.f32,shape=max_num_particles)

particle_num=ti.field(dtype=ti.i32,shape=())

rest_length=ti.field(dtype=ti.f32,shape=(max_num_particles,max_num_particles))

dt=1e-3
substeps=5
springl=ti.field(dtype=ti.f32,shape=())

springy=ti.field(dtype=ti.f32,shape=())

attf=ti.field(dtype=ti.f32,shape=())

drag_damping=ti.field(dtype=ti.f32,shape=())

avf=ti.field(dtype=ti.f32,shape=())


springl[None]=0.1
springy[None] = 1000
attf[None]=10
drag_damping[None]=1
avf[None]=1
particle_num[None]=0



@ti.kernel
def add_particle(pos_x:ti.f32,pos_y:ti.f32,fixed_:ti.i32):
    particle_id=particle_num[None]
    
    x[particle_id]=ti.Vector([pos_x,pos_y])
    fixed[particle_id]=fixed_
    
    for i in range(particle_num[None]):
        if (x[particle_id]-x[i]).norm()<0.15:
            rest_length[i,particle_id]=0.1
            rest_length[particle_id,i]=0.1   
    particle_num[None]+=1
    

@ti.kernel
def substep():
    n=particle_num[None]
    g=ti.Vector([0,-9.8])
    for i in range(n):
        sf=ti.Vector([0,0])
        af=ti.Vector([0,0])
        af+=-v[i]*avf[None]*100
        for j in range(n):
            if rest_length[i,j]>0:
                x_ij=x[i]-x[j]
                fdirect=x_ij.normalized()
                L=x_ij.norm()
                sf+=-springy[None] * (L/rest_length[i,j]-1) * fdirect   #spring f
                #print("sf",sf,rest_length[i,j])
                
                 #air fraction f
                
        v[i]+= dt* (g+sf+af) * (1-fixed[i]) ## if fixed,(1-fixed=0,dv=0)
        #v[i]*=ti.exp(-dt*drag_damping[None])
        x[i]+= dt* v[i]
    
    
        ##collision with wall

        for d in ti.static(range(2)):
            if x[i][d]<0.1:
                x[i][d]=0.1
                v[i][d]=0#-v[i][d]
            if x[i][d]>0.9:
                x[i][d]=0.9
                v[i][d]=0#-v[i][d]

@ti.kernel
def attract(pos_x:ti.f32,pos_y:ti.f32):
    pos=ti.Vector([pos_x,pos_y])
    for i in range(particle_num[None]):
        v[i]+=-dt*substeps*attf[None]*(x[i]-pos)*100*(1-fixed[i])
        v[i]*=ti.exp(-dt*drag_damping[None])*(1-fixed[i])
    
    
        

def main():
    gui=ti.GUI(name='my first game',res=(512,512), background_color = 0xDDDDDD)
    
    while(True):
        #############################
        ### sub step
        for i in range(substeps):
            substep()
        
        #############################
        ### get info form input
        
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key in [ti.GUI.ESCAPE,ti.GUI.EXIT]:
                return
            elif e.key==ti.GUI.LMB:
                print(e.pos[0],e.pos[1])
                add_particle(e.pos[0],e.pos[1],int(gui.is_pressed(ti.GUI.SHIFT)))
                for i in range(particle_num[None]):
                    print(x[i][0],x[i][1])
            elif e.key==ti.GUI.RMB:
                attract(e.pos[0],e.pos[1])
                for i in range(particle_num[None]):
                    for j in range(particle_num[None]):
                        print(rest_length[i,j])
        
        
        
        X=x.to_numpy() ##accelerate
        
        for p in range(particle_num[None]):
            if fixed[p]==0:
                gui.circle(X[p],color=0x0,radius=5)
            else:
                gui.circle(X[p],color=0xDD0000,radius=5)
                

                
        
        ### show spring
        for i in range((particle_num[None])):
            for j in range((particle_num[None])):
                if rest_length[i,j]!=0:
                    gui.line(begin=X[i],end=X[j],color=0x00DD00,radius=2)
                
        #############################
        ### show
        gui.show()

if __name__=="__main__":
    main()

[Taichi] Starting on arch=x64
[Taichi] materializing...
0.490234375 0.236328125
0.490234375 0.236328125
0.546875 0.138671875
0.490234375 0.1201673373579979
0.546875 0.138671875
0.505859375 0.15234375
0.4676068425178528 0.10000000149011612
0.5664799213409424 0.11478584259748459
0.505859375 0.15234375
0.599609375 0.173828125
0.46360108256340027 0.10000000149011612
0.5639793872833252 0.10000000149011612
0.5138425230979919 0.185797780752182
0.599609375 0.173828125
0.556640625 0.220703125
0.45909249782562256 0.10000000149011612
0.5589929223060608 0.10000000149011612
0.5104383230209351 0.1859636902809143
0.6110964417457581 0.18391010165214539
0.556640625 0.220703125
0.390625 0.185546875
0.4396306872367859 0.10061297565698624
0.5386961102485657 0.10000000149011612
0.5066668391227722 0.1766694039106369
0.6181361079216003 0.12761430442333221
0.587395191192627 0.20798614621162415
0.390625 0.185546875
0.517578125 0.2421875
0.4366838037967682 0.10000000149011612
0.5355637669563293 0.10000000149011